<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/09_text_classification_with_Transformer_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 言語モデルを使ったテキスト分類
* 今回はトランスフォーマ言語モデルのファインチューニングを実践する。
  * パラメータ数は数億個オーダのもの。
  * パラメータ数が数十億個（数ビリオン）のものは、扱いがやや大変。
* ファインチューニングによってテキスト分類の性能を向上させる。

* 今回はTransformersライブラリを使う。
  * Sentence Transformersは使わない。

* ランタイムのタイプをGPUに設定しておく。

## インストール

In [ ]:
!pip install -U accelerate datasets evaluate

## 準備

In [ ]:
from transformers import set_seed
set_seed(0)

## データセット
* ライブドアニュースコーパスを使う。
* 前々回に作成したtraining/validation/testのsliceを使う。
  * https://github.com/tomonari-masada/course2024-nlp/blob/main/livedoor_ds.tar.gz

In [ ]:
!wget https://github.com/tomonari-masada/course2024-nlp/raw/refs/heads/main/livedoor_ds.tar.gz
!tar zxf livedoor_ds.tar.gz

In [ ]:
from datasets import load_from_disk
ds = load_from_disk("livedoor_ds")

In [ ]:
ds

In [ ]:
set(ds["train"]["category"])

In [ ]:
category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

In [ ]:
num_labels = len(set(ds["train"]["category"]))
num_labels

In [ ]:
ds["train"]["content"][0]

## トークナイザ

* E5の多言語版を使う。
  * https://arxiv.org/abs/2212.03533
  * https://arxiv.org/abs/2402.05672

* テキストのトークン化をここで済ませておく。

In [ ]:
from transformers import AutoTokenizer

model_name = "intfloat/multilingual-e5-large-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
  return tokenizer(
    examples["content"],
    padding="max_length",
    truncation=True,
    return_tensors="pt",
  )

tokenized_ds = ds.map(tokenize_function, batched=True)

* 正解ラベルのカラムを"label"にrenameする。

In [ ]:
for slice in tokenized_ds:
  tokenized_ds[slice] = tokenized_ds[slice].rename_column("category", "label")

In [ ]:
train_ds = tokenized_ds["train"]
eval_ds = tokenized_ds["validation"]

In [ ]:
train_ds

## 言語モデル

* `AutoModelForSequenceClassification`クラスを使う。

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
).to(0)

* モデルの中身を見てみる。
  * `classifier`というモジュールに注目。

In [ ]:
model

* 分類器として使えることを確認する。

In [ ]:
input = tokenize_function(tokenized_ds["test"][0]).to(model.device)
input

In [ ]:
model.eval()
with torch.no_grad():
  logits = model(**input).logits
model.train()
logits

In [ ]:
import torch
torch.argmax(model(**input).logits, axis=-1)

* ファインチューニング前の分類性能を見てみる。

In [ ]:
from tqdm.auto import tqdm

def evaluate(model, ds, eval_batch_size=1):
    model.eval()
    predicted_class_ids = []
    offset = 0
    for offset in tqdm(range(0, len(ds), eval_batch_size)):
        examples = ds[offset:offset + eval_batch_size]
        input = tokenize_function(examples).to(model.device)
        with torch.no_grad():
            logits = model(**input).logits
        predicted_class_ids.append(torch.argmax(logits, axis=-1))
    model.train()
    return torch.concat(predicted_class_ids)

predicted_class_ids = evaluate(model, ds["test"], eval_batch_size=32)

In [ ]:
print(predicted_class_ids)

In [ ]:
print(ds["test"]["category"])

* 当然、ランダムな予測の分類性能に近い。
  * 分類用のヘッドが全くtrainingされていないから。

In [ ]:
import evaluate
metric = evaluate.load("accuracy")
metric.compute(predictions=predicted_class_ids, references=ds["test"]["category"])

## 評価を実行するヘルパ関数

* logitsと正解ラベルを渡せばaccuracyを返してくれる関数。

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

## `Trainer`の作成

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3, #もっと増やしてもいい。
    eval_strategy="steps",
    eval_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
for name, _ in model.named_parameters():
  print(name)

* ここでは、classifier headと最後の4層だけをtrainingする。
  * 時間とGPUメモリの節約のため。

In [ ]:
for name, param in model.named_parameters():
  if "classifier" in name:
    param.requires_grad = True
  elif ("roberta.encoder.layer.2" in name
        and "roberta.encoder.layer.2." not in name):
    param.requires_grad = True
  else:
    param.requires_grad = False

## ファインチューニングの実行

* CUDA out of memoryエラーが出たら・・・
  * バッチサイズを小さくするなど、GPUのメモリを使わない工夫をしてから・・・
  * セッションを再起動して最初のセルからやり直す。


In [ ]:
trainer.train()

In [ ]:
trainer.save_model("my_model")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("my_model").to(0)

In [ ]:
from tqdm.auto import tqdm

def evaluate(model, ds, eval_batch_size=1):
    model.eval()
    predicted_class_ids = []
    offset = 0
    for offset in tqdm(range(0, len(ds), eval_batch_size)):
        examples = ds[offset:offset + eval_batch_size]
        input = tokenize_function(examples).to(model.device)
        with torch.no_grad():
            logits = model(**input).logits
        predicted_class_ids.append(torch.argmax(logits, axis=-1))
    model.train()
    return torch.concat(predicted_class_ids)

predicted_class_ids = evaluate(model, ds["test"], eval_batch_size=32)

In [ ]:
metric.compute(predictions=predicted_class_ids, references=ds["test"]["category"])